- modify mistake
- implement permutation importance FE, null importance FE

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from hyperopt import hp, tpe, Trials, fmin, space_eval
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        #return -mod_qwk(y, X_p, weights=weights)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(0.4,0.5), np.random.uniform(0.5,0.6), np.random.uniform(0.6,0.7)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead') #Powell
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

In [4]:
def eval_qwk_lgb_regr(y_pred, train_t):
    dist = Counter(train_t['accuracy_group'])
    for k in dist:
        dist[k] /= len(train_t)
    
    acum = 0
    bound = {}
    for i in range(3):
        acum += dist[i]
        bound[i] = np.percentile(y_pred, acum * 100)

    def classify(x):
        if x <= bound[0]:
            return 0
        elif x <= bound[1]:
            return 1
        elif x <= bound[2]:
            return 2
        else:
            return 3

    y_pred = np.array(list(map(classify, y_pred)))
    
    return y_pred

# install

In [5]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 22s, sys: 11.4 s, total: 1min 33s
Wall time: 1min 33s


# Preprocess and Feature engineering

In [6]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = sorted(list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique()))))
    
    train['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), train['type'], train['world']))
    test['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), test['type'], test['world']))
    list_of_type_world = sorted(list(set(train['type_world'].unique()).union(set(test['type_world'].unique()))))
    
    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
        
    train["true"] = train["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)
    test["true"] = test["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)

    train["false"] = train["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    test["false"] = test["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    
    train["game_complete"] = train["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    test["game_complete"] = test["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    
    #train["level"] = train["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    #test["level"] = test["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    
    #train["round"] = train["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
    #test["round"] = test["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
               
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world = encode_title(train, test)

CPU times: user 1min 59s, sys: 12.4 s, total: 2min 11s
Wall time: 2min 11s


In [7]:
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    event_code_count = {str(ev): 0 for ev in list_of_event_code}
    title_count = {actv: 0 for actv in list_of_user_activities}
    type_world_count = {str(ev): 0 for ev in list_of_type_world}
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    last_game_time_title = {'lgt_' + title: 0 for title in assess_titles}
    ac_game_time_title = {'agt_' + title: 0 for title in assess_titles}
    ac_true_attempts_title = {'ata_' + title: 0 for title in assess_titles}
    ac_false_attempts_title = {'afa_' + title: 0 for title in assess_titles}
    
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = user_sample.iloc[0]['timestamp']
    miss = 0
    crys_game_true = 0; crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    durations = []
    prev_assess_title = -999
    assess_count = 1
    last_accuracy = -999
    prev_assess_start = -999; prev_assess_end = -999
    real_prev_assess_start = -999; real_prev_assess_end = -999
    real_assess_start = -999; real_assess_end = -999
    complete_games = 0
    no_result_count = 0
    crys_game_level = np.array([]); tree_game_level = np.array([]); magma_game_level = np.array([])
    crys_game_round = np.array([]); tree_game_round = np.array([]); magma_game_round = np.array([])
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        if session_type != 'Assessment':
            if session_type == "Game":
                true = session['true'].sum()
                false = session['false'].sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #crys_game_level = np.concatenate([crys_game_level, session["level"]], axis=0)
                    #crys_game_round = np.concatenate([crys_game_round, session["round"]], axis=0)
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #tree_game_level = np.concatenate([tree_game_level, session["level"]], axis=0)
                    #tree_game_round = np.concatenate([tree_game_round, session["round"]], axis=0)
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #magma_game_level = np.concatenate([magma_game_level, session["level"]], axis=0)
                    #magma_game_round = np.concatenate([magma_game_round, session["round"]], axis=0)
                else:
                    pass
                
        if (session_type == 'Assessment') & (test_set or len(session)>1): 
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            assess_start = session.iloc[0,2]
            assess_end = session.iloc[-1,2]
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features.update(event_code_count.copy())
            features.update(title_count.copy())
            features.update(type_world_count.copy())
            features.update(last_accuracy_title.copy())
            features.update(last_game_time_title.copy())
            features.update(ac_game_time_title.copy())
            features.update(ac_true_attempts_title.copy())
            features.update(ac_false_attempts_title.copy())
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            ac_true_attempts_title['ata_' + session_title_text] += true_attempts
            ac_false_attempts_title['afa_' + session_title_text] += false_attempts
            last_game_time_title['lgt_' + session_title_text] = session['game_time'].iloc[-1]
            ac_game_time_title['agt_' + session_title_text] += session['game_time'].iloc[-1]
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            features["no_complete_game"] = complete_games
            features["no_result_count"] = no_result_count 
            
            if true_attempts + false_attempts == 0:
                no_result_count += 1
            else:
                real_assess_start = session.iloc[0,2]
                real_assess_end = session.iloc[-1,2]

            if session_world == activities_world["CRYSTALCAVES"]:
                features["game_true"] = crys_game_true
                features["game_false"] = crys_game_false
                features['game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
                #features["hightest_level"] = np.nanmax(crys_game_level) if len(crys_game_level[~np.isnan(crys_game_level)]) >=1 else -1
                #features["level_count"] = len(crys_game_level[~np.isnan(crys_game_level)])
                #features["round_count"] = len(crys_game_round[~np.isnan(crys_game_round)])
            elif session_world == activities_world["TREETOPCITY"]:
                features["game_true"] = tree_game_true
                features["game_false"] = tree_game_false
                features['game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
                #features["hightest_level"] = np.nanmax(tree_game_level) if len(tree_game_level[~np.isnan(tree_game_level)]) >=1 else -1
                #features["level_count"] = len(tree_game_level[~np.isnan(tree_game_level)])
                #features["round_count"] = len(tree_game_round[~np.isnan(tree_game_round)])
            elif session_world == activities_world["MAGMAPEAK"]:
                features["game_true"] = magma_game_true
                features["game_false"] = magma_game_false
                features['game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
                #features["hightest_level"] = np.nanmax(magma_game_level) if len(magma_game_level[~np.isnan(magma_game_level)]) >=1 else -1
                #features["level_count"] = len(magma_game_level[~np.isnan(magma_game_level)])
                #features["round_count"] = len(magma_game_round[~np.isnan(magma_game_round)])
            
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            features["prev_assess_title"] = prev_assess_title
            prev_assess_title = session_title
            features["first_assessment"] = 1 if assess_count == 1 else 0
            assess_count += 1
            features["time_from_start"] = (assess_start - time_first_activity).seconds

            if prev_assess_end == -999:
                features["time_bet_assess"] = -999
            else:
                features["time_bet_assess"] = (assess_start - prev_assess_end).seconds
            prev_assess_start = assess_start
            prev_assess_end = assess_end
            if real_prev_assess_end == -999:
                features["time_bet_real_assess"] = -999
            else:
                features["time_bet_real_assess"] = (real_assess_start - real_prev_assess_end).seconds
            real_prev_assess_start = real_assess_start
            real_prev_assess_end = real_assess_end
            
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
                features['duration_std'] = 0
                features['duration_max'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                features['duration_max'] = np.max(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds)
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            features['last_assess_acc'] = last_accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            last_accuracy = accuracy
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
            
        complete_games += np.sum(session["game_complete"])
        miss += np.sum(session["misses"])
        user_world_count["world_"+str(session_world)] += session.shape[0]
        
        n_of_type_world = Counter(session['type_world']) 
        for key in n_of_type_world.keys():
            type_world_count[str(key)] += n_of_type_world[key]
            
        n_of_title = Counter(session['title']) 
        for key in n_of_title.keys():
            title_count[activities_labels[key]] += n_of_title[key]
            
        n_of_eventcode = Counter(session['event_code']) 
        for key in n_of_eventcode.keys():
            event_code_count[str(key)] += n_of_eventcode[key]
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1], all_assessments[:-1] # test previous data to incorporate into training
    return all_assessments

In [8]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val = []

    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
        compiled_train += get_data(user_sample)
    del train
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
        test_data, val_data = get_data(user_sample, test_set=True)
        compiled_test.append(test_data)
        compiled_val += val_data
    del test
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)

    categoricals = ['session_title']
    return reduce_train, reduce_test, reduce_val, categoricals
new_train, new_test, new_val, categoricals = get_train_and_test(train, test)

In [9]:
tmp = new_train[new_train.Game==0].copy()
tmp = tmp[tmp.Activity == 0].copy()
tmp = tmp[tmp.Clip == 0].copy()
tmp = tmp[tmp.Assessment ==0].copy()
remove_train_index = tmp.index
new_train = new_train[~new_train.index.isin(remove_train_index)].copy()

In [10]:
print(new_train.shape)
print(new_test.shape)
print(new_val.shape)

(17577, 563)
(1000, 563)
(2347, 563)


# Feature selection

In [11]:
def exclude(reduce_train, reduce_test, features):
    to_exclude = [] 
    ajusted_test = reduce_test.copy()
    for feature in features:
        if feature not in ['accuracy_group', 'installation_id', 'session_title', 'hightest_level']:
            data = reduce_train[feature]
            train_mean = data.mean()
            data = ajusted_test[feature] 
            test_mean = data.mean()
            try:
                ajust_factor = train_mean / test_mean
                if ajust_factor > 10 or ajust_factor < 0.1:# or error > 0.01:
                    to_exclude.append(feature)
                    print(feature)
                else:
                    ajusted_test[feature] *= ajust_factor
            except:
                to_exclude.append(feature)
                print(feature)
    return to_exclude, ajusted_test
features = [i for i in new_train.columns if i not in ["game_session"]]
to_exclude, ajusted_test = exclude(new_train, new_test, features)

Air Show_4080
Bottle Filler (Activity)_2010
Bubble Bath_4080
Bubble Bath_4090
Bug Measurer (Activity)_4080
Cart Balancer (Assessment)_4080
Chest Sorter (Assessment)_4080
Crystals Rule_2010
Dino Dive_4080
Dino Drink_4080
Egg Dropper (Activity)_4080
Fireworks (Activity)_4080
Happy Camel_4080
Leaf Leader_4080
Mushroom Sorter (Assessment)_4080
Mushroom Sorter (Assessment)_4090
Pan Balance_2010
Pan Balance_4080
Sandcastle Builder (Activity)_2010
Scrub-A-Dub_4080
Watering Hole (Activity)_2010
acc_Cart Balancer (Assessment)


In [12]:
# permutation importance
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict

def permuted(df):
    for column_name in df.columns:
        permuted_df = df.copy()
        permuted_df[column_name] = np.random.permutation(permuted_df[column_name])
        yield column_name, permuted_df


def pimp(clf, X, y, cv=None, eval_func=roc_auc_score):
    base_scores = []
    permuted_scores = defaultdict(list)

    if cv is None:
        #cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv = GroupKFold(n_splits=5)
        
    for train_index, test_index in cv.split(X, y, X["installation_id"]):
        # 学習用データと検証用データに分割する
        X_train2, y_train2 = X.iloc[train_index], y.iloc[train_index]
        X_test2, y_test2 = X.iloc[test_index], y.iloc[test_index]

        # 学習用データでモデルを学習する
        clf.fit(X_train2, y_train2)

        # まずは何もシャッフルしていないときのスコアを計算する
        y_pred_base = clf.predict(X_test2)
        base_score = eval_func(y_test2, y_pred_base)
        base_scores.append(base_score)

        # 特定のカラムをシャッフルした状態で推論したときのスコアを計算する
        permuted_X_test_gen = permuted(X_test2)
        for column_name, permuted_X_test in permuted_X_test_gen:
            y_pred_permuted = clf.predict(permuted_X_test)
            permuted_score = eval_func(y_test2, y_pred_permuted)
            permuted_scores[column_name].append(permuted_score)

    # 基本のスコアとシャッフルしたときのスコアを返す
    np_base_score = np.array(base_scores)
    dict_permuted_score = {name: np.array(scores) for name, scores in permuted_scores.items()}
    return np_base_score, dict_permuted_score

def score_difference_statistics(base, permuted):
    mean_base_score = base.mean()
    for column_name, scores in permuted.items():
        score_differences = scores - mean_base_score
        yield column_name, score_differences.mean(), score_differences.std()

# prepare for the data ---
X_train = new_train.drop(['accuracy_group'],axis=1) 
y_train = new_train.accuracy_group.copy()
y_train.loc[y_train <=1] = 0
y_train.loc[y_train >=2] = 1
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = [i for i in X_train.columns if i in to_exclude]
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]    

# execute permutation importance 
clf = RandomForestClassifier(n_estimators=100)
base_score, permuted_scores = pimp(clf, X_train, y_train)
diff_stats = list(score_difference_statistics(base_score, permuted_scores))
pimp_df = pd.DataFrame(diff_stats, columns = ["feature", "score_mean", "score_std"])
pimp_df = pimp_df.sort_values("score_mean", ascending=True).reset_index(drop=True)
pimp_df.head(10)

,feature,score_mean,score_std
0,session_title,-0.046460,0.004225
1,last_game_acc,-0.003497,0.007074
2,Crystal Caves - Level 3_2000,-0.003429,0.005911
3,prev_assess_title,-0.002796,0.004488
4,accumulated_accuracy_group,-0.002579,0.008982
5,time_bet_assess,-0.002306,0.005730
6,game_true,-0.002035,0.006498
7,Assessment_MAGMAPEAK,-0.001857,0.007149
8,Cart Balancer (Assessment)_4100,-0.001630,0.006027
9,time_from_start,-0.001629,0.004627


In [13]:
# null importance
# https://www.kaggle.com/ogrellier/feature-selection-with-null-importances
def get_feature_importances(data, shuffle, target, id_var, seed=None):
    # Gather real features
    train_features = [f for f in data if f not in [target, id_var]]
    # Go over fold and keep track of CV score (train and valid) and feature importances
    
    # Shuffle target if required
    y = data[target].copy()
    if shuffle:
        # Here you could as well use a binomial distribution
        y = data[target].copy().sample(frac=1.0)
    
    categorical_feats = ["session_title"]
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'rf',
        'subsample': 0.623,
        'colsample_bytree': 0.7,
        'num_leaves': 127,
        'max_depth': 8,
        'seed': seed,
        'bagging_freq': 1,
        'n_jobs': 4
    }
    
    # Fit the model
    clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=200, categorical_feature=categorical_feats)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(train_features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    imp_df['trn_score'] = roc_auc_score(y, clf.predict(data[train_features]))
    
    return imp_df

def build_null_df(data, target, id_var):
    import time
    null_imp_df = pd.DataFrame()
    nb_runs = 80
    start = time.time()
    dsp = ''
    for i in range(nb_runs):
        # Get current run importances
        imp_df = get_feature_importances(data=data, target=target, id_var=id_var, shuffle=True)
        imp_df['run'] = i + 1 
        # Concat the latest importances with the old ones
        null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
        # Erase previous message
        for l in range(len(dsp)):
            print('\b', end='', flush=True)
        # Display current run and time used
        spent = (time.time() - start) / 60
        dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
        print(dsp, end='', flush=True)
    return null_imp_df

def calculate_imp(actual_imp_df, null_imp_df):
    feature_scores = []
    for _f in actual_imp_df['feature'].unique():
        f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
        f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
        gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
        f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
        f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
        split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
        feature_scores.append((_f, split_score, gain_score))

    scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])
    return scores_df

# prepare for the data
np.random.seed(123)
train_df = new_train.copy()
train_df.accuracy_group.loc[train_df.accuracy_group <=1] = 0
train_df.accuracy_group.loc[train_df.accuracy_group >=2] = 1

# execute null importance
actual_imp_df = get_feature_importances(train_df, shuffle=False, target="accuracy_group", id_var = "installation_id", seed=123)
null_imp_df = build_null_df(data=train_df, target="accuracy_group", id_var = "installation_id")
scores_df = calculate_imp(actual_imp_df, null_imp_df)

Done with    1 of   80 (Spent   0.2 min)Done with    2 of   80 (Spent   0.4 min)Done with    3 of   80 (Spent   0.5 min)Done with    4 of   80 (Spent   0.7 min)Done with    5 of   80 (Spent   0.9 min)Done with    6 of   80 (Spent   1.0 min)Done with    7 of   80 (Spent   1.2 min)Done with    8 of   80 (Spent   1.4 min)Done with    9 of   80 (Spent   1.6 min)Done with   10 of   80 (Spent   1.7 min)Done with   11 of   80 (Spent   1.9 min)Done with   12 of   80 (Spent   2.1 min)Done with   13 of   80 (Spent   2.3 min)

In [14]:
def corr_calculate_imp(actual_imp_df, null_imp_df):
    correlation_scores = []
    for _f in actual_imp_df['feature'].unique():
        f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
        f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].values
        gain_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
        f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
        f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].values
        split_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
        correlation_scores.append((_f, split_score, gain_score))

    corr_scores_df = pd.DataFrame(correlation_scores, columns=['feature', 'split_score', 'gain_score'])
    return corr_scores_df
corr_scores_df = corr_calculate_imp(actual_imp_df, null_imp_df)

In [15]:
def score_feature_selection(df=None, train_features=None, cat_feats=None, target=None):
    # Fit LightGBM 
    dtrain = lgb.Dataset(df[train_features], target, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': .1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'num_leaves': 31,
        'max_depth': -1,
        'seed': 13,
        'n_jobs': 4,
        'min_split_gain': .00001,
        'reg_alpha': .00001,
        'reg_lambda': .00001,
        'metric': 'auc'
    }
    
    # Fit the model
    hist = lgb.cv(
        params=lgb_params, 
        train_set=dtrain, 
        num_boost_round=2000,
        categorical_feature=cat_feats,
        nfold=5,
        stratified=True,
        shuffle=True,
        early_stopping_rounds=50,
        verbose_eval=0,
        seed=17
    )
    # Return the last mean / std values 
    return hist['auc-mean'][-1], hist['auc-stdv'][-1]

categorical_feats = ["session_title"]
for threshold in [0, 10, 20, 30 , 40, 50 ,60 , 70, 80 , 90, 95, 99]:
    split_feats = sorted(list(corr_scores_df[corr_scores_df.split_score >= threshold]["feature"]))
    split_cat_feats = sorted(list(set(corr_scores_df[corr_scores_df.split_score >= threshold]["feature"]) & set(categorical_feats)))
    gain_feats = sorted(list(corr_scores_df[corr_scores_df.gain_score >= threshold]["feature"]) )
    gain_cat_feats = sorted(list(set(corr_scores_df[corr_scores_df.gain_score >= threshold]["feature"]) & set(categorical_feats)))
                                                                                             
    print('Results for threshold %3d' % threshold)
    split_results = score_feature_selection(df=train_df, train_features=split_feats, cat_feats=split_cat_feats, target=train_df['accuracy_group'])
    print('\t SPLIT : %.6f +/- %.6f' % (split_results[0], split_results[1]))
    gain_results = score_feature_selection(df=train_df, train_features=gain_feats, cat_feats=gain_cat_feats, target=train_df['accuracy_group'])
    print('\t GAIN  : %.6f +/- %.6f' % (gain_results[0], gain_results[1]))

Results for threshold   0
	 SPLIT : 0.831613 +/- 0.008996
	 GAIN  : 0.831613 +/- 0.008996
Results for threshold  10
	 SPLIT : 0.831520 +/- 0.008201
	 GAIN  : 0.832110 +/- 0.009035
Results for threshold  20
	 SPLIT : 0.832069 +/- 0.007369
	 GAIN  : 0.832512 +/- 0.009771
Results for threshold  30
	 SPLIT : 0.833748 +/- 0.009030
	 GAIN  : 0.832049 +/- 0.009323
Results for threshold  40
	 SPLIT : 0.832629 +/- 0.009572
	 GAIN  : 0.832330 +/- 0.009571
Results for threshold  50
	 SPLIT : 0.832750 +/- 0.008536
	 GAIN  : 0.832602 +/- 0.007724
Results for threshold  60
	 SPLIT : 0.832769 +/- 0.009414
	 GAIN  : 0.832319 +/- 0.008316
Results for threshold  70
	 SPLIT : 0.831842 +/- 0.009055
	 GAIN  : 0.832466 +/- 0.009186
Results for threshold  80
	 SPLIT : 0.832186 +/- 0.008802
	 GAIN  : 0.833393 +/- 0.009017
Results for threshold  90
	 SPLIT : 0.832527 +/- 0.008716
	 GAIN  : 0.833009 +/- 0.008128
Results for threshold  95
	 SPLIT : 0.831620 +/- 0.008747
	 GAIN  : 0.832304 +/- 0.009451
Results fo

In [16]:
#scores_df.sort_values("gain_score", ascending = False).reset_index(drop=True)

# classification

In [17]:
from catboost import CatBoost
from catboost import Pool
# https://catboost.ai/docs/concepts/python-reference_catboost_predict.html
# https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html

In [18]:
def feature_selection(train):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    cat_params = {
    'loss_function': 'Logloss',
    'depth': 4,
    "num_boost_round":100000,
    'learning_rate': 0.01,
    "early_stopping_rounds":10,}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    features_list = [i for i in X_train.columns if i != "installation_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
            
        train_pool = Pool(X_train2, label=y_train2)
        test_pool = Pool(X_test2, label=y_test2)
        clf = CatBoost(cat_params)
        clf.fit(train_pool, eval_set=[train_pool, test_pool], use_best_model=True, verbose_eval = 300)
        test_predict = clf.predict(X_test2, prediction_type = "Probability")
        valid.iloc[test_index] = test_predict[:,1].reshape(X_test2.shape[0], 1)
        feature_importance_df["Fold_"+str(i+1)] = clf.get_feature_importance()
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    return feature_importance_df
df_for_classification = feature_selection(new_train)

Fold 1
0:	learn: 0.6899418	test: 0.6899418	test1: 0.6901186	best: 0.6901186 (0)	total: 98.5ms	remaining: 2h 44m 10s
300:	learn: 0.5109190	test: 0.5109190	test1: 0.5224478	best: 0.5224478 (300)	total: 13.3s	remaining: 1h 13m 18s
600:	learn: 0.4887568	test: 0.4887568	test1: 0.5037096	best: 0.5037096 (600)	total: 26.6s	remaining: 1h 13m 26s
900:	learn: 0.4768182	test: 0.4768182	test1: 0.4956070	best: 0.4956070 (900)	total: 39.5s	remaining: 1h 12m 22s
1200:	learn: 0.4638097	test: 0.4638097	test1: 0.4910713	best: 0.4910433 (1198)	total: 52.2s	remaining: 1h 11m 33s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.4906463515
bestIteration = 1241

Shrink model to first 1242 iterations.
Fold 2
0:	learn: 0.6900306	test: 0.6900306	test1: 0.6900942	best: 0.6900942 (0)	total: 43.2ms	remaining: 1h 12m 1s
300:	learn: 0.5115867	test: 0.5115867	test1: 0.5197172	best: 0.5197172 (300)	total: 12.9s	remaining: 1h 11m 26s
600:	learn: 0.4892546	test: 0.4892546	test1: 0.5031422	best: 0.5031

In [19]:
def accuracy_class(train, test, fea, select_flg):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    X_test = X_test[sorted(X_test.columns.tolist())]
    if select_flg == True:
        X_test = X_test[fea]
    print(X_test.shape[1])
    
    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    cat_params = {
    'loss_function': 'Logloss',
    'depth': 4,
    "num_boost_round":100000,
    'learning_rate': 0.01,
    "early_stopping_rounds":100,}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
        if select_flg == True:
            X_train2 = X_train2[fea] 
            X_test2 = X_test2[fea]
            
        train_pool = Pool(X_train2, label=y_train2)
        test_pool = Pool(X_test2, label=y_test2)
        clf = CatBoost(cat_params)
        clf.fit(train_pool, eval_set=[train_pool, test_pool], use_best_model=True, verbose_eval = 300)
        test_predict = clf.predict(X_test2, prediction_type = "Probability")
        
        models.append(clf)
        valid.iloc[test_index] = test_predict[:,1].reshape(X_test2.shape[0], 1)
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    print('Accuracy score = \t {}'.format(accuracy_score(y_train, np.round(valid))))
    print('Precision score = \t {}'.format(precision_score(y_train, np.round(valid))))
    print('Recall score =   \t {}'.format(recall_score(y_train, np.round(valid))))
    print('F1 score =      \t {}'.format(f1_score(y_train, np.round(valid))))
    print(confusion_matrix(y_train, np.round(valid)))
    pred_value = np.zeros([X_test.shape[0]])
    for model in models:
        pred_value += model.predict(X_test, prediction_type = "Probability")[:,1] / len(models)
    return pred_value, valid

tmp = df_for_classification.sort_values("Cv", ascending = True).reset_index(drop=True).copy()
feat = tmp[tmp.index <= 120]["Feature"]
pred_value, valid = accuracy_class(new_train, new_test, feat, False)

537
Fold 1
0:	learn: 0.6899418	test: 0.6899418	test1: 0.6901186	best: 0.6901186 (0)	total: 44.3ms	remaining: 1h 13m 54s
300:	learn: 0.5109190	test: 0.5109190	test1: 0.5224478	best: 0.5224478 (300)	total: 13.1s	remaining: 1h 12m 9s
600:	learn: 0.4887568	test: 0.4887568	test1: 0.5037096	best: 0.5037096 (600)	total: 26.2s	remaining: 1h 12m 6s
900:	learn: 0.4768182	test: 0.4768182	test1: 0.4956070	best: 0.4956070 (900)	total: 39.8s	remaining: 1h 12m 53s
1200:	learn: 0.4638097	test: 0.4638097	test1: 0.4910713	best: 0.4910433 (1198)	total: 52.6s	remaining: 1h 12m 10s
1500:	learn: 0.4526600	test: 0.4526600	test1: 0.4890793	best: 0.4890793 (1500)	total: 1m 5s	remaining: 1h 11m 26s
1800:	learn: 0.4430370	test: 0.4430370	test1: 0.4879908	best: 0.4879908 (1800)	total: 1m 18s	remaining: 1h 10m 57s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.4877186947
bestIteration = 1925

Shrink model to first 1926 iterations.
Fold 2
0:	learn: 0.6900306	test: 0.6900306	test1: 0.6900942	be

In [20]:
best_score = 0
for i in range(10):
    optR = OptimizedRounder()
    optR.fit(np.array(valid).reshape(-1,), new_train.accuracy_group, random_flg=True)
    coefficients = optR.coefficients()
    final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
    score = qwk(new_train.accuracy_group, final_valid_pred)
    print(i, np.sort(coefficients), score)
    if score > best_score:
        best_score = score
        best_coefficients = coefficients
final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(best_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(test_exp_accuracy).reshape(-1,), [-np.inf] + list(np.sort([0.30278718, 0.42545025, 0.56396292])) + [np.inf], labels = [0, 1, 2, 3])

sample_submission["accuracy_group"] = final_test_pred.astype(int)
sample_submission.to_csv('submission.csv', index=False)
sample_submission["accuracy_group"].value_counts(normalize = True)

0 [0.35956246 0.52791334 0.74837277] 0.59014357
1 [0.38723163 0.5291244  0.74916191] 0.59048583
2 [0.3733441  0.51992717 0.74992837] 0.58997539
3 [0.3746515  0.55209225 0.74994618] 0.58963858
4 [0.3867983  0.5499634  0.74895577] 0.58980931
5 [0.37664827 0.52984829 0.74965131] 0.59020788
6 [0.38696398 0.5497763  0.74842302] 0.58989818
7 [0.38587198 0.54652947 0.74916004] 0.58999843
8 [0.3331923  0.52789184 0.74838247] 0.59034532
9 [0.37307028 0.5090121  0.74966794] 0.58968999


3    0.468
2    0.265
0    0.184
1    0.083
Name: accuracy_group, dtype: float64